![ewatercycle logo](https://raw.githubusercontent.com/eWaterCycle/ewatercycle/main/docs/examples/logo.png)

# PCRGlobWB example use case: CMIP6

This example shows how the [PCRGlobWB model](https://globalhydrology.nl/research/models/pcr-globwb-2-0/) can be used within the eWaterCycle system. It is assumed you have already seen [this tutorial notebook](../../example_model_run_HBV.ipynb) explaining how to run the simple HBV model for the River Leven at Newby Bridge. 

The PCRGlobWB model is an example of a distributed model where fluxes and stores in the balance are calculated for grid cells (often also called pixels). This requires both the forcing data as well as any parameters to also be spatially distributed. Depending on the complexity of the model, these datasets can be quite large in memory size.

Here we will be running PCRGLobWB for Great Brittain and will extract discharge data at the location of the River Leven again, to compare with the HBV model run. We will also demonstrate how to interact with the state of the model, during runtime, showcasing the benefit of using the BMI interface when building experiments using models.

In [1]:
# This cell is only used to suppress some distracting output messages
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
import matplotlib.pyplot as plt
from cartopy import crs
from cartopy import feature as cfeature
import fiona
import shapely.geometry
from pyproj import Geod
from rich import print
import pandas as pd
import xarray as xr

import ewatercycle.forcing
import ewatercycle.models
import ewatercycle.parameter_sets

In [3]:
station_latitude = 32.76108802182168 #Gila near Yuma
station_longitude = -114.41721725761596

## Loading a parameter set

For this example we have prepared and hosted a global parameter set made by Utrecht University. For each model run, what needs to be specified to deliniate the region of interest is a "clone map". The config file has many options, one of which is the location of this clone map.

Note that this is very specific to PCRGlobWB. For complex (and legacy) models like PCRGlobWB one needs to know quite detailed information about the model before being able to run it. However, using eWaterCycle does reduce the time for seting up the model and getting it to run.

In [4]:
parameter_set = ewatercycle.parameter_sets.ParameterSet(
    name="custom_parameter_set",
    directory= "/data/shared/parameter-sets/pcrglobwb_global",
    config="./pcrglobwb_sonoran_mpi-esm1-2-hr_05min.ini",
    target_model="pcrglobwb",
    supported_model_versions={"setters"},
)

In [5]:
print(parameter_set)

ParameterSet(
    name='custom_parameter_set',
    directory=PosixPath('/data/shared/parameter-sets/pcrglobwb_global'),
    config=PosixPath('pcrglobwb_sonoran_mpi-esm1-2-hr_05min.ini'),
    doi='N/A',
    target_model='pcrglobwb',
    supported_model_versions={'setters'},
    downloader=None
)

## Load forcing data

For this example case, the forcing is generated in [this seperate notebook](generate_forcing.ipynb). This is a common practice when generating forcing takes considerable (CPU, memory, disk) resources. 

In the cell below, we load the pre-generated forcing. Note that in contrast with HBV, PCRGlobWB only needs temperature and precipitation as forcing inputs. HBV also needs potential evaporation. PCRGlobWB calculated potential and actual evaporation as part of its update step.

In [6]:
forcing = ewatercycle.forcing.sources["PCRGlobWBForcing"].load(
    directory="/home/emol/forcing/csonoran_desert/work/diagnostic/script",
)
print(forcing)

PermissionError: [Errno 13] Permission denied: '/home/emol/forcing/csonoran_desert/work/diagnostic/script/ewatercycle_forcing.yaml'

## Setting up the model

Note that the model version and the parameterset versions should be compatible.

In [ ]:
pcrglob = ewatercycle.models.PCRGlobWB(
    parameter_set=parameter_set,
    forcing=forcing
)
print(pcrglob)

In [ ]:
pcrglob.version

eWaterCycle exposes a selected set of configurable parameters. These can be modified in the `setup()` method.

In [ ]:
print(pcrglob.parameters)

Calling `setup()` will start up the model container. Be careful with calling it multiple times!

In [ ]:
cfg_file, cfg_dir = pcrglob.setup(
    end_time="2025-08-31T00:00:00Z",
    max_spinups_in_years=0
)
cfg_file, cfg_dir

In [ ]:
print(pcrglob.parameters)

Note that the parameters have been changed. A new config file which incorporates these updated parameters has been generated as well. If you want to see or modify any additional model settings, you can acces this file directly. When you're ready, pass the path to the config file to `initialize()`.

In [ ]:
pcrglob.initialize(cfg_file)

We prepare a small dataframe where we can store the discharge output from the model

In [ ]:
time = pd.date_range(pcrglob.start_time_as_isostr, pcrglob.end_time_as_isostr)
timeseries = pd.DataFrame(
    index=pd.Index(time, name="time"), columns=["PCRGlobWB: Sonoran Desert"]
)
timeseries.head()

## Running the model

Simply running the model from start to end is straightforward. At each time step we can retrieve information from the model.

In [ ]:
while pcrglob.time < pcrglob.end_time:
    pcrglob.update()

    # Track discharge at station location
    discharge_at_station = pcrglob.get_value_at_coords(
        "groundwater recharge", lat=[station_latitude], lon=[station_longitude]
    )
    time = pcrglob.time_as_isostr
    timeseries["PCRGlobWB: Sonoran Desert"][time] = discharge_at_station[0]

    # Show progress
    print(time,end='\r')  # "\r" clears the output before printing the next timestamp


## Interacting with the model

PCRGlobWB exposes many variables. Just a few of them are shown here:

In [ ]:
#list(pcrglob.output_var_names)[-15:-5]

Model fields can be fetched as xarray objects (or as flat numpy arrays using `get_value()`):

In [ ]:
da = pcrglob.get_value_as_xarray("groundwater recharge")
da.thin(5)  # only show every 5th value in each dim

Xarray makes it very easy to plot the data. In the figure below, we add a cross at the location where we collected the discharge every timestep: Leven at Newby bridge.

In [ ]:
fig = plt.figure(dpi=120)
ax = fig.add_subplot(111, projection=crs.PlateCarree())
da.plot(ax=ax, cmap="GnBu")

# Overlay ocean and coastines
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.RIVERS, color="k")
ax.coastlines()

# Add a red cross marker at the location of the Leven River at Newby Bridge
ax.scatter(station_longitude, station_latitude, s=250, c="r", marker="x", lw=2)

We can get (or set) the values at custom points as well:

In [ ]:
#extra
timeseries.plot()

We of course want to compare this both to observations as well as to the result of the HBV model.


In [ ]:
prepared_forcing_path_caravan_central = "/home/emol/forcing/caravan/camelsgb_73010"
caravan_forcing = ewatercycle.forcing.sources['CaravanForcing'].load(directory="/home/rhut/forcing/caravan/hysets_09488650")
xr_caravan_forcing = xr.open_dataset(caravan_forcing['Q'])


In [ ]:
timeseries.plot()
xr_caravan_forcing["Q"].plot(label="Observed discharge")

plt.legend()

## Cleaning up

Models usually perform some "wrap up tasks" at the end of a model run, such as writing the last outputs to disk and releasing memory. In the case of eWaterCycle, another important teardown task is destroying the container in which the model was running. This can free up a lot of resources on your system. Therefore it is good practice to always call `finalize()` when you're done with an experiment.

In [ ]:
pcrglob.finalize()